In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [37]:
import re
import csv
from datetime import datetime
from collections import namedtuple

%matplotlib inline
import matplotlib.pyplot as plt


# pip install workalendar
# https://github.com/novafloss/workalendar
from workalendar.europe import Italy

In [39]:
cal = Italy()

def def_is_holiday(dt_obj):
    if(cal.is_working_day(dt_obj)):
        return 0
    else:
        return 1

In [40]:
row = namedtuple("sample", ['sottoarea', 'area', 'zona', 'prodotto', 'data', 'timestamp', 'giorno_della_settimana', 'giorno_del_mese',
                              'giorno_dell_anno', 'mese', 'anno', 'festivo','vendite'])

In [41]:
def do_parse(x):
    # Oggetto Datetime
    dt_obj = datetime.strptime(str(x[4]), '"%Y-%m-%d"')
    
    return row(
        sottoarea = int(re.sub("[^0-9]", "", x[2])),
        area = int(re.sub("[^0-9]", "", x[1])),
        zona = int(re.sub("[^0-9]", "", str(x[0]))),  
        prodotto = int(re.sub("[^0-9]", "", x[3])),
        data = x[4],
        timestamp = int(dt_obj.strftime("%s")),
        giorno_della_settimana = int(dt_obj.weekday()),
        giorno_del_mese = int(dt_obj.weekday()),
        giorno_dell_anno = int(dt_obj.timetuple().tm_yday),
        mese = int(dt_obj.strftime("%m")),
        anno = int(dt_obj.strftime("%Y")),
        festivo = def_is_holiday(dt_obj),
        vendite = int(x[5])
        )

def parse(sc,filepath):
    rawRdd = sc.textFile(filepath)
    
    header = rawRdd.first()

    # Togliamo l'header
    rawRdd = rawRdd.filter(lambda x: x != header)

    # Togliamo le righe vuote
    rawRdd = rawRdd.filter(lambda x: x != "")
    
    return rawRdd.map(lambda x: x.split(",")).map(lambda x: do_parse(x))

In [42]:
rdd = parse(sc,"dataset/dataset_polimi.csv")
rdd.first()

sample(sottoarea=5, area=1, zona=1, prodotto=1, data='"2014-01-01"', timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0)

# Pandas
if you cannot stay without it

In [43]:
import pandas as pd

In [44]:
#porto sottoarea come prima colonna per poi fare il join
#rdd = rdd.map(lambda x: (x[2], x[1], x[0], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11]))

In [45]:
row1 = namedtuple("gps", ['sottoarea', 'lat', 'lon', 'citta', 'provincia', 'regione', 'zona_italia',
                              'isola', 'abitanti', 'min_distanza', 'densita'])

In [46]:
def do_parse_locations(x):
    return row1(
        sottoarea = int(x[0]),
        lat = float(x[1]),
        lon = float(x[2]),  
        citta = x[3],
        provincia = x[4],
        regione = x[5],
        zona_italia = x[6],
        isola = int(x[7]),
        abitanti = int(x[8]),
        min_distanza = int(x[9]),
        densita = int(x[10])
        )

def parse_locations(sc,filepath):
    rawRdd = sc.textFile(filepath)
    
    header = rawRdd.first()

    # Togliamo l'header
    rawRdd = rawRdd.filter(lambda x: x != header)
    
    return rawRdd.map(lambda x: x.split(",")).map(lambda x: do_parse_locations(x))

In [47]:
rdd.first()

sample(sottoarea=5, area=1, zona=1, prodotto=1, data='"2014-01-01"', timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0)

In [48]:
rdd1 = rdd.map(lambda x: (x[0], list(x[1:])))
rdd1.first()

(5, [1, 1, 1, '"2014-01-01"', 1388530800, 2, 2, 1, 1, 2014, 1, 0])

In [49]:
rdd_gps = parse_locations(sc,"dataset/gps_processed_andrea.csv")
rdd_gps.first()

gps(sottoarea=1, lat=37.57251433333334, lon=14.20335411764705, citta='Villarosa', provincia='EN', regione='SIC', zona_italia='Sud', isola=1, abitanti=4937, min_distanza=50450, densita=0)

In [50]:
rdd_gps1 = rdd_gps.map(lambda x: (x[0], list(x[1:])))
rdd_gps1.first()

(1,
 [37.57251433333334,
  14.20335411764705,
  'Villarosa',
  'EN',
  'SIC',
  'Sud',
  1,
  4937,
  50450,
  0])

In [51]:
rdd.filter(lambda x: x.sottoarea == 68).take(10)

[sample(sottoarea=68, area=55, zona=20, prodotto=1, data='"2014-01-01"', timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0),
 sample(sottoarea=68, area=55, zona=20, prodotto=1, data='"2014-01-02"', timestamp=1388617200, giorno_della_settimana=3, giorno_del_mese=3, giorno_dell_anno=2, mese=1, anno=2014, festivo=0, vendite=1),
 sample(sottoarea=68, area=55, zona=20, prodotto=1, data='"2014-01-03"', timestamp=1388703600, giorno_della_settimana=4, giorno_del_mese=4, giorno_dell_anno=3, mese=1, anno=2014, festivo=0, vendite=1),
 sample(sottoarea=68, area=55, zona=20, prodotto=1, data='"2014-01-04"', timestamp=1388790000, giorno_della_settimana=5, giorno_del_mese=5, giorno_dell_anno=4, mese=1, anno=2014, festivo=1, vendite=1),
 sample(sottoarea=68, area=55, zona=20, prodotto=1, data='"2014-01-05"', timestamp=1388876400, giorno_della_settimana=6, giorno_del_mese=6, giorno_dell_anno=5, mese=1, anno=2014, festivo=1, ve

In [53]:
result = namedtuple("sample", ['sottoarea', 'area', 'zona', 'lat', 'lon', 'CE', 'PD', 'FC', 
                                'PR', 'CH', 'SA', 'PG', 'BS', 'FE', 'VI', 'ME', 'AG', 'TO', 
                                'PV', 'LU', 'VA', 'BG', 'MN', 'SI', 'GE', 'BA', 'RN', 'BL', 
                                'CA', 'VE', 'PU', 'VT', 'BO', 'LC', 'MI', 'TR', 'AT', 'FI', 
                                'CN', 'CO', 'RI', 'FM', 'FG', 'MT', 'IS', 'CS', 'FR', 'CT', 
                                'LE', 'VC', 'SS', 'MB', 'VV', 'OR', 'PC', 'TP', 'NU', 'UD', 
                                'RO', 'BR', 'BZ', 'RE', 'PZ', 'RG', 'PA', 'LO', 'PO', 'SV', 
                                'AR', 'AV', 'BN', 'PN', 'LT', 'TE', 'AL', 'CZ', 'CR', 'TV', 
                                'VR', 'PI', 'AN', 'TA', 'TN', 'RC', 'OT', 'AQ', 'SR', 'RM', 
                                'MO', 'GR', 'EN', 'PT', 'ABR', 'BAS', 'CAL', 'CAM', 'EMR',
                                'FVG', 'LAZ', 'LIG', 'LOM', 'MAR', 'MOL', 'PIE', 'PUG', 'SAR',
                                'SIC', 'TOS', 'TAA', 'UMB', 'VDA', 'VEN', 'nord', 'centro', 
                                'sud', 'isola', 'abitanti', 'min_distanza', 'densita', 
                                'prodotto', 'data', 'timestamp', 'giorno_della_settimana', 'giorno_del_mese',
                                'giorno_dell_anno', 'mese', 'anno', 'festivo', 'vendite'])

In [54]:
def do_parse_join(x):
    return result(
        sottoarea = x[0],
        area = x[1][1][0],
        zona = x[1][1][1],
        lat = x[1][0][0],
        lon = x[1][0][1],
        #provincia = x[1][0][3],
        CE = 1 if x[1][0][3] == "CE" else 0,
        PD = 1 if x[1][0][3] == "PD" else 0,
        FC = 1 if x[1][0][3] == "FC" else 0,
        PR = 1 if x[1][0][3] == "RP" else 0,
        CH = 1 if x[1][0][3] == "CH" else 0,
        SA = 1 if x[1][0][3] == "SA" else 0,
        PG = 1 if x[1][0][3] == "PG" else 0,
        BS = 1 if x[1][0][3] == "BS" else 0,
        FE = 1 if x[1][0][3] == "FE" else 0,
        VI = 1 if x[1][0][3] == "VI" else 0,
        ME = 1 if x[1][0][3] == "ME" else 0,
        AG = 1 if x[1][0][3] == "AG" else 0,
        TO = 1 if x[1][0][3] == "TO" else 0,
        PV = 1 if x[1][0][3] == "PV" else 0,
        LU = 1 if x[1][0][3] == "LU" else 0,
        VA = 1 if x[1][0][3] == "VA" else 0,
        BG = 1 if x[1][0][3] == "BG" else 0,
        MN = 1 if x[1][0][3] == "MN" else 0,
        SI = 1 if x[1][0][3] == "SI" else 0,
        GE = 1 if x[1][0][3] == "GE" else 0,
        BA = 1 if x[1][0][3] == "BA" else 0,
        RN = 1 if x[1][0][3] == "RN" else 0,
        BL = 1 if x[1][0][3] == "BL" else 0,
        CA = 1 if x[1][0][3] == "CA" else 0,
        VE = 1 if x[1][0][3] == "VE" else 0,
        PU = 1 if x[1][0][3] == "PU" else 0,
        VT = 1 if x[1][0][3] == "VT" else 0,
        BO = 1 if x[1][0][3] == "BO" else 0,
        LC = 1 if x[1][0][3] == "LC" else 0,
        MI = 1 if x[1][0][3] == "MI" else 0,
        TR = 1 if x[1][0][3] == "TR" else 0,
        AT = 1 if x[1][0][3] == "AT" else 0,
        FI = 1 if x[1][0][3] == "FI" else 0,
        CN = 1 if x[1][0][3] == "CN" else 0,
        CO = 1 if x[1][0][3] == "CO" else 0,
        RI = 1 if x[1][0][3] == "RI" else 0,
        FM = 1 if x[1][0][3] == "FM" else 0,
        FG = 1 if x[1][0][3] == "FG" else 0,
        MT = 1 if x[1][0][3] == "MT" else 0,
        IS = 1 if x[1][0][3] == "IS" else 0,
        CS = 1 if x[1][0][3] == "CS" else 0,
        FR = 1 if x[1][0][3] == "FR" else 0,
        CT = 1 if x[1][0][3] == "CT" else 0,
        LE = 1 if x[1][0][3] == "LE" else 0,
        VC = 1 if x[1][0][3] == "VC" else 0,
        SS = 1 if x[1][0][3] == "SS" else 0,
        MB = 1 if x[1][0][3] == "MB" else 0,
        VV = 1 if x[1][0][3] == "VV" else 0,
        OR = 1 if x[1][0][3] == "OR" else 0,
        PC = 1 if x[1][0][3] == "PC" else 0,
        TP = 1 if x[1][0][3] == "TP" else 0,
        NU = 1 if x[1][0][3] == "NU" else 0,
        UD = 1 if x[1][0][3] == "UD" else 0,
        RO = 1 if x[1][0][3] == "RO" else 0,
        BR = 1 if x[1][0][3] == "BR" else 0,
        BZ = 1 if x[1][0][3] == "BZ" else 0,
        RE = 1 if x[1][0][3] == "RE" else 0,
        PZ = 1 if x[1][0][3] == "PZ" else 0,
        RG = 1 if x[1][0][3] == "RG" else 0,
        PA = 1 if x[1][0][3] == "PA" else 0,
        LO = 1 if x[1][0][3] == "LO" else 0,
        PO = 1 if x[1][0][3] == "PO" else 0,
        SV = 1 if x[1][0][3] == "SV" else 0,
        AR = 1 if x[1][0][3] == "AR" else 0,
        AV = 1 if x[1][0][3] == "AV" else 0,
        BN = 1 if x[1][0][3] == "BN" else 0,
        PN = 1 if x[1][0][3] == "PN" else 0,
        LT = 1 if x[1][0][3] == "LT" else 0,
        TE = 1 if x[1][0][3] == "TE" else 0,
        AL = 1 if x[1][0][3] == "AL" else 0,
        CZ = 1 if x[1][0][3] == "CZ" else 0,
        CR = 1 if x[1][0][3] == "CR" else 0,
        TV = 1 if x[1][0][3] == "TV" else 0,
        VR = 1 if x[1][0][3] == "VR" else 0,
        PI = 1 if x[1][0][3] == "PI" else 0,
        AN = 1 if x[1][0][3] == "AN" else 0,
        TA = 1 if x[1][0][3] == "TA" else 0,
        TN = 1 if x[1][0][3] == "TN" else 0,
        RC = 1 if x[1][0][3] == "RC" else 0,
        OT = 1 if x[1][0][3] == "OT" else 0,
        AQ = 1 if x[1][0][3] == "AQ" else 0,
        SR = 1 if x[1][0][3] == "SR" else 0,
        RM = 1 if x[1][0][3] == "RM" else 0,
        MO = 1 if x[1][0][3] == "MO" else 0,
        GR = 1 if x[1][0][3] == "GR" else 0,
        EN = 1 if x[1][0][3] == "EN" else 0,
        PT = 1 if x[1][0][3] == "PT" else 0,
        #regione = x[1][0][4],
        ABR = 1 if x[1][0][4] == "ABR" else 0,
        BAS = 1 if x[1][0][4] == "BAS" else 0,
        CAL = 1 if x[1][0][4] == "CAL" else 0,
        CAM = 1 if x[1][0][4] == "CAM" else 0,
        EMR = 1 if x[1][0][4] == "EMR" else 0,
        FVG = 1 if x[1][0][4] == "FVG" else 0,
        LAZ = 1 if x[1][0][4] == "LAZ" else 0,
        LIG = 1 if x[1][0][4] == "LIG" else 0,
        LOM = 1 if x[1][0][4] == "LOM" else 0,
        MAR = 1 if x[1][0][4] == "MAR" else 0,
        MOL = 1 if x[1][0][4] == "MOL" else 0,
        PIE = 1 if x[1][0][4] == "PIE" else 0,
        PUG = 1 if x[1][0][4] == "PUG" else 0,
        SAR = 1 if x[1][0][4] == "SAR" else 0,
        SIC = 1 if x[1][0][4] == "SIC" else 0,
        TOS = 1 if x[1][0][4] == "TOS" else 0,
        TAA = 1 if x[1][0][4] == "TAA" else 0,
        UMB = 1 if x[1][0][4] == "UMB" else 0,
        VDA = 1 if x[1][0][4] == "VDA" else 0,
        VEN = 1 if x[1][0][4] == "VEN" else 0,
        nord = 1 if x[1][0][5] == "Nord" else 0,
        centro = 1 if x[1][0][5] == "Centro" and not (x[1][0][4] == "SAR" or x[1][0][4] == "SIC") else 0,
        sud = 1 if x[1][0][5] == "Sud" and not (x[1][0][4] == "SAR" or x[1][0][4] == "SIC") else 0,
        isola = 1 if x[1][0][4] == "SAR" or x[1][0][4] == "SIC" else 0,
        abitanti = x[1][0][7],
        min_distanza = x[1][0][8],
        densita = x[1][0][9] + 1, #facendo così considero anche se stesso nel raggio, evitando di avere densita 0 che con certi algoritmi potrebbe dare problemi
        prodotto = x[1][1][2],
        data = x[1][1][3].replace('"', ''),
        timestamp = x[1][1][4],
        giorno_della_settimana = x[1][1][5],
        giorno_del_mese = x[1][1][6],
        giorno_dell_anno = x[1][1][7],
        mese = x[1][1][8],
        anno = x[1][1][9],
        festivo = x[1][1][10],
        vendite = x[1][1][11],
    )

In [55]:
res = rdd_gps1.join(rdd1).map(lambda x: do_parse_join(x))
res.first()

sample(sottoarea=68, area=55, zona=20, lat=38.16890584269665, lon=15.005016404494388, CE=0, PD=0, FC=0, PR=0, CH=0, SA=0, PG=0, BS=0, FE=0, VI=0, ME=1, AG=0, TO=0, PV=0, LU=0, VA=0, BG=0, MN=0, SI=0, GE=0, BA=0, RN=0, BL=0, CA=0, VE=0, PU=0, VT=0, BO=0, LC=0, MI=0, TR=0, AT=0, FI=0, CN=0, CO=0, RI=0, FM=0, FG=0, MT=0, IS=0, CS=0, FR=0, CT=0, LE=0, VC=0, SS=0, MB=0, VV=0, OR=0, PC=0, TP=0, NU=0, UD=0, RO=0, BR=0, BZ=0, RE=0, PZ=0, RG=0, PA=0, LO=0, PO=0, SV=0, AR=0, AV=0, BN=0, PN=0, LT=0, TE=0, AL=0, CZ=0, CR=0, TV=0, VR=0, PI=0, AN=0, TA=0, TN=0, RC=0, OT=0, AQ=0, SR=0, RM=0, MO=0, GR=0, EN=0, PT=0, ABR=0, BAS=0, CAL=0, CAM=0, EMR=0, FVG=0, LAZ=0, LIG=0, LOM=0, MAR=0, MOL=0, PIE=0, PUG=0, SAR=0, SIC=1, TOS=0, TAA=0, UMB=0, VDA=0, VEN=0, nord=0, centro=0, sud=0, isola=1, abitanti=13431, min_distanza=33235, densita=1, prodotto=1, data='2014-01-01', timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0)

In [56]:
res.map(lambda x: x.densita).filter(lambda x: x > 0).max()

7

In [57]:
sottoaree = set(rdd.map(lambda x: x.sottoarea).collect())
sottoaree

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145}

In [58]:
collected_rdd = res.collect()

In [59]:
df = pd.DataFrame.from_records(collected_rdd, columns = ['sottoarea', 'area', 'zona', 'lat', 'lon', 'CE', 'PD', 'FC', 
                                                         'PR', 'CH', 'SA', 'PG', 'BS', 'FE', 'VI', 'ME', 'AG', 'TO', 
                                                         'PV', 'LU', 'VA', 'BG', 'MN', 'SI', 'GE', 'BA', 'RN', 'BL', 
                                                         'CA', 'VE', 'PU', 'VT', 'BO', 'LC', 'MI', 'TR', 'AT', 'FI', 
                                                         'CN', 'CO', 'RI', 'FM', 'FG', 'MT', 'IS', 'CS', 'FR', 'CT', 
                                                         'LE', 'VC', 'SS', 'MB', 'VV', 'OR', 'PC', 'TP', 'NU', 'UD', 
                                                         'RO', 'BR', 'BZ', 'RE', 'PZ', 'RG', 'PA', 'LO', 'PO', 'SV', 
                                                         'AR', 'AV', 'BN', 'PN', 'LT', 'TE', 'AL', 'CZ', 'CR', 'TV', 
                                                         'VR', 'PI', 'AN', 'TA', 'TN', 'RC', 'OT', 'AQ', 'SR', 'RM', 
                                                         'MO', 'GR', 'EN', 'PT', 'ABR', 'BAS', 'CAL', 'CAM', 'EMR',
                                                         'FVG', 'LAZ', 'LIG', 'LOM', 'MAR', 'MOL', 'PIE', 'PUG', 'SAR',
                                                         'SIC', 'TOS', 'TAA', 'UMB', 'VDA', 'VEN', 'nord', 'centro', 'sud', 
                                                         'isola', 'abitanti', 'min_distanza', 'densita', 'prodotto', 'data',
                                                         'timestamp', 'giorno_della_settimana', 'giorno_del_mese', 
                                                         'giorno_dell_anno', 'mese', 'anno', 'festivo', 'vendite'])

In [60]:
df[1:10]

,sottoarea,area,zona,lat,lon,CE,PD,FC,PR,CH,...,prodotto,data,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,mese,anno,festivo,vendite
1,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-02,1388617200,3,3,2,1,2014,0,1
2,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-03,1388703600,4,4,3,1,2014,0,1
3,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-04,1388790000,5,5,4,1,2014,1,1
4,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-05,1388876400,6,6,5,1,2014,1,2
5,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-06,1388962800,0,0,6,1,2014,1,0
6,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-07,1389049200,1,1,7,1,2014,0,2
7,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-08,1389135600,2,2,8,1,2014,0,3
8,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-09,1389222000,3,3,9,1,2014,0,3
9,68,55,20,38.168906,15.005016,0,0,0,0,0,...,1,2014-01-10,1389308400,4,4,10,1,2014,0,3


In [61]:
#df[1:10].values

In [24]:
#df.to_csv('dataset_polimi_andrea3.csv', sep=',', index=False)

In [34]:
#df[1:10].sottoarea

In [62]:
for s in sottoaree:
    prod1 = res.filter(lambda x: x.prodotto == 1 and x.sottoarea == s)
    collected_rdd1 = prod1.collect()
    df1 = pd.DataFrame.from_records(collected_rdd1, columns = ['sottoarea', 'area', 'zona', 'lat', 'lon', 'CE', 'PD', 'FC', 
                                                         'PR', 'CH', 'SA', 'PG', 'BS', 'FE', 'VI', 'ME', 'AG', 'TO', 
                                                         'PV', 'LU', 'VA', 'BG', 'MN', 'SI', 'GE', 'BA', 'RN', 'BL', 
                                                         'CA', 'VE', 'PU', 'VT', 'BO', 'LC', 'MI', 'TR', 'AT', 'FI', 
                                                         'CN', 'CO', 'RI', 'FM', 'FG', 'MT', 'IS', 'CS', 'FR', 'CT', 
                                                         'LE', 'VC', 'SS', 'MB', 'VV', 'OR', 'PC', 'TP', 'NU', 'UD', 
                                                         'RO', 'BR', 'BZ', 'RE', 'PZ', 'RG', 'PA', 'LO', 'PO', 'SV', 
                                                         'AR', 'AV', 'BN', 'PN', 'LT', 'TE', 'AL', 'CZ', 'CR', 'TV', 
                                                         'VR', 'PI', 'AN', 'TA', 'TN', 'RC', 'OT', 'AQ', 'SR', 'RM', 
                                                         'MO', 'GR', 'EN', 'PT', 'ABR', 'BAS', 'CAL', 'CAM', 'EMR',
                                                         'FVG', 'LAZ', 'LIG', 'LOM', 'MAR', 'MOL', 'PIE', 'PUG', 'SAR',
                                                         'SIC', 'TOS', 'TAA', 'UMB', 'VDA', 'VEN', 'nord', 'centro', 'sud', 
                                                         'isola', 'abitanti', 'min_distanza', 'densita', 'prodotto', 'data',
                                                         'timestamp', 'giorno_della_settimana', 'giorno_del_mese', 
                                                         'giorno_dell_anno', 'mese', 'anno', 'festivo', 'vendite'])
    df1.to_csv('dataset/prodotto_sottoarea/prodotto1_sottoarea' + str(s) +'.csv', sep=',', index=False)
    prod2 = res.filter(lambda x: x.prodotto == 2 and x.sottoarea == s)
    collected_rdd2 = prod2.collect()
    df2 = pd.DataFrame.from_records(collected_rdd2, columns = ['sottoarea', 'area', 'zona', 'lat', 'lon', 'CE', 'PD', 'FC', 
                                                         'PR', 'CH', 'SA', 'PG', 'BS', 'FE', 'VI', 'ME', 'AG', 'TO', 
                                                         'PV', 'LU', 'VA', 'BG', 'MN', 'SI', 'GE', 'BA', 'RN', 'BL', 
                                                         'CA', 'VE', 'PU', 'VT', 'BO', 'LC', 'MI', 'TR', 'AT', 'FI', 
                                                         'CN', 'CO', 'RI', 'FM', 'FG', 'MT', 'IS', 'CS', 'FR', 'CT', 
                                                         'LE', 'VC', 'SS', 'MB', 'VV', 'OR', 'PC', 'TP', 'NU', 'UD', 
                                                         'RO', 'BR', 'BZ', 'RE', 'PZ', 'RG', 'PA', 'LO', 'PO', 'SV', 
                                                         'AR', 'AV', 'BN', 'PN', 'LT', 'TE', 'AL', 'CZ', 'CR', 'TV', 
                                                         'VR', 'PI', 'AN', 'TA', 'TN', 'RC', 'OT', 'AQ', 'SR', 'RM', 
                                                         'MO', 'GR', 'EN', 'PT', 'ABR', 'BAS', 'CAL', 'CAM', 'EMR',
                                                         'FVG', 'LAZ', 'LIG', 'LOM', 'MAR', 'MOL', 'PIE', 'PUG', 'SAR',
                                                         'SIC', 'TOS', 'TAA', 'UMB', 'VDA', 'VEN', 'nord', 'centro', 'sud', 
                                                         'isola', 'abitanti', 'min_distanza', 'densita', 'prodotto', 'data',
                                                         'timestamp', 'giorno_della_settimana', 'giorno_del_mese', 
                                                         'giorno_dell_anno', 'mese', 'anno', 'festivo', 'vendite'])
    df2.to_csv('dataset/prodotto_sottoarea/prodotto2_sottoarea' + str(s) +'.csv', sep=',', index=False)
    print(s)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145


In [27]:
prod1 = res.filter(lambda x: x.prodotto == 1 and x.sottoarea == 68)
prod1.first()

sample(sottoarea=68, area=55, zona=20, lat=38.16890584269665, lon=15.005016404494388, CE=0, PD=0, FC=0, PR=0, CH=0, SA=0, PG=0, BS=0, FE=0, VI=0, ME=1, AG=0, TO=0, PV=0, LU=0, VA=0, BG=0, MN=0, SI=0, GE=0, BA=0, RN=0, BL=0, CA=0, VE=0, PU=0, VT=0, BO=0, LC=0, MI=0, TR=0, AT=0, FI=0, CN=0, CO=0, RI=0, FM=0, FG=0, MT=0, IS=0, CS=0, FR=0, CT=0, LE=0, VC=0, SS=0, MB=0, VV=0, OR=0, PC=0, TP=0, NU=0, UD=0, RO=0, BR=0, BZ=0, RE=0, PZ=0, RG=0, PA=0, LO=0, PO=0, SV=0, AR=0, AV=0, BN=0, PN=0, LT=0, TE=0, AL=0, CZ=0, CR=0, TV=0, VR=0, PI=0, AN=0, TA=0, TN=0, RC=0, OT=0, AQ=0, SR=0, RM=0, MO=0, GR=0, EN=0, PT=0, ABR=0, BAS=0, CAL=0, CAM=0, EMR=0, FVG=0, LAZ=0, LIG=0, LOM=0, MAR=0, MOL=0, PIE=0, PUG=0, SAR=0, SIC=1, TOS=0, TAA=0, UMB=0, VDA=0, VEN=0, nord=0, centro=0, sud=0, isola=1, abitanti=13431, min_distanza=33235, densita=1, prodotto=1, data='2014-01-01', timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0)

In [28]:
collected_rdd1 = prod1.collect()

In [29]:
df1 = pd.DataFrame.from_records(collected_rdd1, columns = ['sottoarea', 'area', 'zona', 'lat', 'lon', 'CE', 'PD', 'FC', 
                                                         'PR', 'CH', 'SA', 'PG', 'BS', 'FE', 'VI', 'ME', 'AG', 'TO', 
                                                         'PV', 'LU', 'VA', 'BG', 'MN', 'SI', 'GE', 'BA', 'RN', 'BL', 
                                                         'CA', 'VE', 'PU', 'VT', 'BO', 'LC', 'MI', 'TR', 'AT', 'FI', 
                                                         'CN', 'CO', 'RI', 'FM', 'FG', 'MT', 'IS', 'CS', 'FR', 'CT', 
                                                         'LE', 'VC', 'SS', 'MB', 'VV', 'OR', 'PC', 'TP', 'NU', 'UD', 
                                                         'RO', 'BR', 'BZ', 'RE', 'PZ', 'RG', 'PA', 'LO', 'PO', 'SV', 
                                                         'AR', 'AV', 'BN', 'PN', 'LT', 'TE', 'AL', 'CZ', 'CR', 'TV', 
                                                         'VR', 'PI', 'AN', 'TA', 'TN', 'RC', 'OT', 'AQ', 'SR', 'RM', 
                                                         'MO', 'GR', 'EN', 'PT', 'ABR', 'BAS', 'CAL', 'CAM', 'EMR',
                                                         'FVG', 'LAZ', 'LIG', 'LOM', 'MAR', 'MOL', 'PIE', 'PUG', 'SAR',
                                                         'SIC', 'TOS', 'TAA', 'UMB', 'VDA', 'VEN', 'nord', 'centro', 'sud', 
                                                         'isola', 'abitanti', 'min_distanza', 'densita', 'prodotto', 'data',
                                                         'timestamp', 'giorno_della_settimana', 'giorno_del_mese', 
                                                         'giorno_dell_anno', 'mese', 'anno', 'festivo', 'vendite'])

In [30]:
df1.to_csv('dataset/prodotto_sottoarea/prodotto1_sottoarea68.csv', sep=',', index=False)

In [29]:
prod2 = res.filter(lambda x: x.prodotto == 2 and x.sottoarea == 68)
prod2.first()

sample(sottoarea=68, area=55, zona=20, lat=38.16890584269665, lon=15.005016404494388, CE=0, PD=0, FC=0, PR=0, CH=0, SA=0, PG=0, BS=0, FE=0, VI=0, ME=1, AG=0, TO=0, PV=0, LU=0, VA=0, BG=0, MN=0, SI=0, GE=0, BA=0, RN=0, BL=0, CA=0, VE=0, PU=0, VT=0, BO=0, LC=0, MI=0, TR=0, AT=0, FI=0, CN=0, CO=0, RI=0, FM=0, FG=0, MT=0, IS=0, CS=0, FR=0, CT=0, LE=0, VC=0, SS=0, MB=0, VV=0, OR=0, PC=0, TP=0, NU=0, UD=0, RO=0, BR=0, BZ=0, RE=0, PZ=0, RG=0, PA=0, LO=0, PO=0, SV=0, AR=0, AV=0, BN=0, PN=0, LT=0, TE=0, AL=0, CZ=0, CR=0, TV=0, VR=0, PI=0, AN=0, TA=0, TN=0, RC=0, OT=0, AQ=0, SR=0, RM=0, MO=0, GR=0, EN=0, PT=0, ABR=0, BAS=0, CAL=0, CAM=0, EMR=0, FVG=0, LAZ=0, LIG=0, LOM=0, MAR=0, MOL=0, PIE=0, PUG=0, SAR=0, SIC=1, TOS=0, TAA=0, UMB=0, VDA=0, VEN=0, nord=0, centro=0, sud=0, isola=1, abitanti=13431, min_distanza=33235, densita=1, prodotto=2, data='"2014-01-01"', timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0)

In [30]:
collected_rdd2 = prod2.collect()

In [31]:
df2 = pd.DataFrame.from_records(collected_rdd2, columns = ['sottoarea', 'area', 'zona', 'lat', 'lon', 'CE', 'PD', 'FC', 
                                                         'PR', 'CH', 'SA', 'PG', 'BS', 'FE', 'VI', 'ME', 'AG', 'TO', 
                                                         'PV', 'LU', 'VA', 'BG', 'MN', 'SI', 'GE', 'BA', 'RN', 'BL', 
                                                         'CA', 'VE', 'PU', 'VT', 'BO', 'LC', 'MI', 'TR', 'AT', 'FI', 
                                                         'CN', 'CO', 'RI', 'FM', 'FG', 'MT', 'IS', 'CS', 'FR', 'CT', 
                                                         'LE', 'VC', 'SS', 'MB', 'VV', 'OR', 'PC', 'TP', 'NU', 'UD', 
                                                         'RO', 'BR', 'BZ', 'RE', 'PZ', 'RG', 'PA', 'LO', 'PO', 'SV', 
                                                         'AR', 'AV', 'BN', 'PN', 'LT', 'TE', 'AL', 'CZ', 'CR', 'TV', 
                                                         'VR', 'PI', 'AN', 'TA', 'TN', 'RC', 'OT', 'AQ', 'SR', 'RM', 
                                                         'MO', 'GR', 'EN', 'PT', 'ABR', 'BAS', 'CAL', 'CAM', 'EMR',
                                                         'FVG', 'LAZ', 'LIG', 'LOM', 'MAR', 'MOL', 'PIE', 'PUG', 'SAR',
                                                         'SIC', 'TOS', 'TAA', 'UMB', 'VDA', 'VEN', 'nord', 'centro', 'sud', 
                                                         'isola', 'abitanti', 'min_distanza', 'densita', 'prodotto', 'data',
                                                         'timestamp', 'giorno_della_settimana', 'giorno_del_mese', 
                                                         'giorno_dell_anno', 'mese', 'anno', 'festivo', 'vendite'])

In [32]:
df2.to_csv('dataset/prodotto_sottoarea/prodotto2_sottoarea68.csv', sep=',', index=False)